<a href="https://colab.research.google.com/github/ajayaram92/Paper-Testing/blob/main/Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importing Modules

In [14]:
import requests
import pandas as pd
import numpy as np
import tensorflow as tf
from scipy import stats
from scipy.signal import savgol_filter

from sklearn.cross_decomposition import PLSRegression
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import mean_squared_error, r2_score


from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import MinMaxScaler

Getting Data from GitHub

In [3]:
def get_data(url):
  df = pd.read_csv(url)
  df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
  return df

Pre Processing function for Sav Gol Fiter and Standardising

In [4]:
url="https://raw.githubusercontent.com/ajayaram92/Paper-Testing/main/ABv1.csv"
urltest = "https://raw.githubusercontent.com/ajayaram92/Paper-Testing/4e49e5fde3d27b7a24d8fa9cd3b5e76ca8b9db3f/peach_spectra_brix.csv"
df = get_data(url)
testdf = get_data(urltest)
print(get_data(url))

      LATITUDE  LONGITUDE    OC   NDVI_v1    s2_1    s2_2    s2_3  s2_4  \
0    12.868275  75.239625  0.51  0.108786   730.0  1022.0  1456.0  2533   
1    12.853780  74.972660  0.79  0.138859  1214.5  1320.0  1963.0  2400   
2    12.853512  74.974800  1.41  0.150655  1202.5  1468.0  2082.0  2478   
3    12.890035  75.230001  0.58  0.093620   740.0  1488.0  2016.0  2447   
4    12.853595  74.972970  0.46  0.147096  1174.0  1447.0  2029.0  2357   
..         ...        ...   ...       ...     ...     ...     ...   ...   
139  12.854510  74.978510  0.86  0.296267   616.5   842.5  1032.5  1348   
140  12.850010  75.230010  0.91  0.202997   802.5   896.5  1089.0  1319   
141  12.849920  74.984800  1.04  0.224261  1065.0   968.5  1303.0  1594   
142  12.877939  75.248664  0.41  0.284812   607.0   630.5   859.5  1135   
143  12.848580  74.983820  0.85  0.289938   818.5  1050.0  1494.0  1502   

       s2_5    s2_6    s2_7    s2_8    s2_9   s2_10   s2_11   s2_12  
0    2725.5  2899.5  3021.5  

In [8]:
def pre_processing(data, sav_gol=True, sav_win=11, sav_pol=2, std=True):
  new_df =  data.drop(["LATITUDE", "LONGITUDE", "OC"], axis =1)  # removing lat long and oc for Sav gol
  if "NDVI_v1" in data.columns:
    new_df = new_df.drop("NDVI_v1", axis =1 )
  

  if sav_gol==True:   # checking if sav_gol is required
    new_df = pd.DataFrame(savgol_filter(new_df, sav_win, sav_pol, axis=1))
  
  
  if "NDVI_v1" in data.columns:
    new_df.columns = data.columns[4:]
  else:
    new_df.columns = data.columns[3:]
  
  new_df["OC"] = data["OC"]
  if std==True:       # checking if standardising is required
      mean = new_df.mean(axis = 0)
      std = new_df.std(axis = 0)
      new_df = (new_df-mean)/std
  
  return (new_df)

In [9]:
pro_df = pre_processing(df)

       s2_1    s2_2    s2_3  s2_4    s2_5    s2_6    s2_7    s2_8    s2_9  \
0     730.0  1022.0  1456.0  2533  2725.5  2899.5  3021.5  3142.0  3067.5   
1    1214.5  1320.0  1963.0  2400  2727.5  3010.0  3301.5  3174.0  3407.0   
2    1202.5  1468.0  2082.0  2478  2888.0  3149.0  3402.5  3281.0  3539.5   
3     740.0  1488.0  2016.0  2447  2407.0  2808.5  3002.0  2952.5  2965.5   
4    1174.0  1447.0  2029.0  2357  2713.5  3053.0  3237.0  3170.0  3365.0   
..      ...     ...     ...   ...     ...     ...     ...     ...     ...   
139   616.5   842.5  1032.5  1348  1617.5  2090.5  2370.0  2483.0  2631.5   
140   802.5   896.5  1089.0  1319  1617.0  2398.5  2832.5  2713.0  3060.0   
141  1065.0   968.5  1303.0  1594  2204.5  2764.5  3200.0  3286.0  3624.0   
142   607.0   630.5   859.5  1135  1509.5  1803.0  2138.5  2347.5  2506.5   
143   818.5  1050.0  1494.0  1502  2198.0  2693.0  2996.0  3117.0  3237.5   

      s2_10   s2_11   s2_12  
0    3153.0  4701.5  3389.0  
1    3267.0  45

In [10]:
print(df.mean(axis=0))
print(pre_processing(df))

LATITUDE       12.920555
LONGITUDE      74.974315
OC              0.970139
NDVI_v1         0.218766
s2_1          786.725694
s2_2         1110.531250
s2_3         1492.836806
s2_4         1919.486111
s2_5         2182.961806
s2_6         2627.638889
s2_7         2908.791667
s2_8         2958.017361
s2_9         3149.187500
s2_10        3103.295139
s2_11        3878.663194
s2_12        2815.208333
dtype: float64
       s2_1    s2_2    s2_3  s2_4    s2_5    s2_6    s2_7    s2_8    s2_9  \
0     730.0  1022.0  1456.0  2533  2725.5  2899.5  3021.5  3142.0  3067.5   
1    1214.5  1320.0  1963.0  2400  2727.5  3010.0  3301.5  3174.0  3407.0   
2    1202.5  1468.0  2082.0  2478  2888.0  3149.0  3402.5  3281.0  3539.5   
3     740.0  1488.0  2016.0  2447  2407.0  2808.5  3002.0  2952.5  2965.5   
4    1174.0  1447.0  2029.0  2357  2713.5  3053.0  3237.0  3170.0  3365.0   
..      ...     ...     ...   ...     ...     ...     ...     ...     ...   
139   616.5   842.5  1032.5  1348  1617.5  209

PLSR

In [11]:
def PLSR(data):
    y = data["OC"]
    X = data.drop("OC", axis=1)
    n_comp= len(data.columns)
    if n_comp > 100:
      n_comp = 20
    mse = []
    component = np.arange(1, n_comp)
 
    for i in component:
        pls = PLSRegression(n_components=i)
 
        # Cross-validation
        y_cv = cross_val_predict(pls, X, y, cv=10)
        mse.append(r2_score(y, y_cv))



    # Calculate minimum in MSE
    print(mse)
    msemin = np.argmax(mse)
    print("number of components used", msemin+1)
 
    # Define PLS object with optimal number of components
    pls_opt = PLSRegression(n_components=msemin+1)
 
    # Fir to the entire dataset
    pls_opt.fit(X, y)
    y_c = pls_opt.predict(X)
 
    # Cross-validation
    y_cv = cross_val_predict(pls_opt, X, y, cv=10)
 
    # Calculate scores for calibration and cross-validation
    score_c = r2_score(y, y_c)
    score_cv = r2_score(y, y_cv)
 
    # Calculate mean squared error for calibration and cross validation
    mse_c = mean_squared_error(y, y_c)
    mse_cv = mean_squared_error(y, y_cv)
 
    print('R2 calib: %5.3f'  % score_c)
    print('R2 CV: %5.3f'  % score_cv)
    print('MSE calib: %5.3f' % mse_c)
    print('MSE CV: %5.3f' % mse_cv)
 
    # Plot regression and figures of merit
    rangey = max(y) - min(y)
    rangex = max(y_c) - min(y_c)
    #z = np.polyfit(y, y_c, 1)

    return score_cv, mse_cv

In [22]:
print(PLSR(pro_df))
#print(PLSR(testdf))

[-0.08058051080929629, -0.019888000821133733, -0.025590192937867018, -0.05074292585687701, -0.032641544006957623, -1.0700573697954603, -0.8792987083328805, -4.378084055681853, -5.0636768178235645, -62.99510912111578, -97.87035235740544, -60006.314654660455]
number of components used 2
R2 calib: 0.051
R2 CV: -0.020
MSE calib: 0.942
MSE CV: 1.013
(-0.019888000821133733, 1.0128054452598758)


Random Forrest Regressor

In [20]:
def RFR(data):
    y = data["OC"]
    X = data.drop("OC", axis=1)
# Perform Grid-Search
    gsc = GridSearchCV(
        estimator=RandomForestRegressor(),
        param_grid={
            'max_depth': range(3,7),
            'n_estimators': (10, 50, 100, 1000),
        },
        cv=5, scoring='neg_mean_squared_error', verbose=0, n_jobs=-1)
    
    grid_result = gsc.fit(X, y)
    best_params = grid_result.best_params_
    
    rfr = RandomForestRegressor(max_depth=best_params["max_depth"], n_estimators=best_params["n_estimators"],random_state=False, verbose=False)
# Perform 10-Fold CV
    scores = cross_val_score(rfr, X, y, cv=10, scoring='neg_mean_absolute_error')
    y_cv = cross_val_predict(rfr, X, y, cv=10)

    score_cv = r2_score(y, y_cv)
    mse_cv = mean_squared_error(y, y_cv)
    return score_cv, mse_cv

In [21]:
print(RFR(pro_df))

(-0.14594822311611244, 1.137990249344473)
